# Chapter 4. Linear Algebra

Linear algebra is the branch of mathematics that deals with vector spaces.  
Intimidated? Go to [Khan Academy](https://www.khanacademy.org/math/linear-algebra) for some lessons.

In [126]:
from __future__ import division # want 3 / 2 == 1.5
import re, math, random # regexes, math functions, random numbers
import matplotlib.pyplot as plt # pyplot
from collections import defaultdict, Counter
from functools import partial

## Vectors

In [127]:
# a three-dimensional vector:
height_weight_age = [70,   # inches
                     170,  # pounds,
                     40,]  # years
height_weight_age

[70, 170, 40]

In [128]:
# a four-dimensional vector:
grades = [95,   # exam1
          80,   # exam2
          75,   # exam3
          62,]  # exam4
grades

[95, 80, 75, 62]

Python lists are <em>not</em> vectors!

In [129]:
v = [1, 2]
w = [2, 1]
v + w  # Python will concatenate these two lists

[1, 2, 2, 1]

###  Vector Addition and Subtraction

Vectors add componentwise.  
Assuming that vectors <code>v</code> and <code>w</code> are the same length, their sum is a vector whose first element is v[0] + w[0], second element is v[1] + w[1], and so on.

In [130]:
# We can add vectors by zip-ing the vectors together and using a list comprehension to add the elements
v = [1, 2]
w = [2, 1]
def vector_add(v, w):
    """ adds corresponding elements """
    return [v_i + w_i for v_i, w_i in zip(v, w)]

vector_add(v, w)

[3, 3]

In [131]:
# We can subtract two vectors by subtracting the corresponding elements
def vector_subtract(v, w):
    """ subtracts corresponding elements """
    return [v_i - w_i for v_i, w_i in zip(v, w)]

vector_subtract(v, w)

[-1, 1]

Neat, huh?  
Here let's sum a list of vectors componentwise.  
We will create a new vector whose first element is the sum of all first elements, second element is the sum of all second elements, and so on.

In [132]:
# The easiest way to do this is by adding one vector at a time
vectors = [[1,2,3],
           [4,5,6],
           [7,8,9]]

def vector_sum(vectors):
    """ sums all corresponding elements """
    result = vectors[0]                      # start with the first vector 
    for vector in vectors[1:]:               # then loop over the others
        result = vector_add(result, vector)  # and add them to the result
    return result

vector_sum(vectors)

[12, 15, 18]

If you think about it, we are just <code>reduce</code>-ing the list of vectors using <code>vector-add</code>, which means we can rewrite this more briefly using higher order functions:

In [133]:
def vector_sum(vectors):
    return reduce(vector_add, vectors)

vector_sum(vectors)

[12, 15, 18]

or even:

In [134]:
# this is probably more clever than helpful, but for demonstrative purposes...
vector_sum = partial(reduce, vector_add)
vector_sum(vectors)

[12, 15, 18]

### Vector Multiplication

We can multiply a vector by a scalar by multiplying each element of the vector by that number.

In [135]:
c = 2
v = [1,2,3,4]

def scalar_multiply(c, v):
    """ c is a number, v is a vector """
    return [c * v_i for v_i in v]

scalar_multiply(c,v)

[2, 4, 6, 8]

Now we can compute the componentwise means of a list of vectors.  
Again, the vectors must be the same size(length).

In [136]:
vectors = [[1,2,3],
           [4,5,6],
           [7,8,9]]

def vector_mean(vectors):
    """ compute the vector whose ith element is the mean of the ith element of the input vectors """
    n = len(vectors)
    return scalar_multiply(1/n, vector_sum(vectors))

vector_mean(vectors)

[4.0, 5.0, 6.0]

The dot product of two vectors is the sum of their componentwise products.  
In other words, the dot product measures how far the vector <code>v</code> extends in the <code>w</code> direction.  
In other words, it's the length of the vector you would get if you <em>projected</em> <code>v</code> onto <code>w</code>.

In [137]:
v = [1,2,3]
w = [4,5,6]

def dot(v, w):
    """ v_1 * w_1 + ... + v_n * w_n """
    return sum(v_i * w_i for v_i, w_i in zip(v, w))

dot(v, w)

32

You can then use dot product to compute a vector's <em>sum of squares<em>.

In [138]:
def sum_of_squares(v):
    """ v_1 * v_1 + ... + v_n * v_n """
    return dot(v, v)

sum_of_squares(v)

14

Now we can compute a vector's <em>magnitude</em> (aka length from tail to tip).  
[Khan Academy](https://www.khanacademy.org/math/precalculus/vectors-precalc/magnitude-vectors/v/finding-vector-magnitude-from-components) has a good explanation.

In [139]:
def magnitude(v):
    return math.sqrt(sum_of_squares(v))  # math.sqrt == square root

magnitude(v)

3.7416573867739413

Now we have all of the pieces we need to compute the distance between two vectors, defined as:  

$\sqrt{(v_1 - w_1)^2 +\;...\;+ (v_n - w_n)^2}$

In [140]:
v = [1,2,3]
w = [4,5,6]

def squared_distance(v, w):
    """ (v_1 - w_1) ** 2 + ... + (v_n - w_n) ** 2 """
    return sum_of_squares(vector_subtract(v, w))

print("Squared distance is " + str(squared_distance(v, w)))

def distance(v, w):
    return math.sqrt(squared_distance(v, w))

distance(v, w)

Squared distance is 27


5.196152422706632

That might be expressed a bit more succinctly as:

In [141]:
def distance(v, w):
    return magnitude(vector_subtract(v, w))

distance(v, w)

5.196152422706632

NOTE:  
Using lists as vectors is great for exposition but terrible for performance.  
In production code, you would want to use the NumPy library, which includes a high-performance array class with all sorts of arithmetic operations included.

## Matrices

A matrix is a two-dimensional collection of numbers.  
We will represent matrices as lists of lists, with each inner list having the same size and representing a row of the matrix.

In [142]:
# A has 2 rows and 3 columns
A = [[1, 2, 3],
     [4, 5, 6]]

# B has 3 rows and 2 columns
B = [[1, 2],
     [3, 4],
     [5, 6]]

Given this list-of-lists representation, the matrix A has len(A) rows and len(A[0]) columns, which we consider its shape:

In [143]:
def shape(A):
    num_rows = len(A)
    num_cols = len(A[0]) if A else 0  # number of elements in the first row 
    return num_rows, num_cols

shape(A)

(2, 3)

If a matrix has n rows and k columns, we will refer to it as a <code>n x k</code> matrix, where each row is a vector of length k and each column is a vector of length n.

In [144]:
def get_row(A, i):
    return A[i]  # A[i] is already the ith row

get_row(A, 0)

[1, 2, 3]

In [145]:
def get_column(A, j):
    return [A_i[j] for A_i in A]  # jth element of row A_i for each row A_i

get_column(A, 0)

[1, 4]

We also want to be able to create a matrix given its shape and a function for generating it elements.  
We can do this using a nested list comprehension:

In [146]:
def make_matrix(num_rows, num_cols, entry_fn):
    """ returns a num_rows by num_cols matrix whose (i, j)th entry is entry_fn(i, j) """
    return [[entry_fn(i, j)  # given i, create a list 
            for j in range(num_cols)]  # [entry_fn(i, 0), ... ]
            for i in range(num_rows)]  # create one list for each i

Given this function, you can make a 5 by 5 identity matrix with:

In [147]:
def is_diagonal(i, j):
    """ ones on the diagonal, zeros everywhere else """
    return 1 if i == j else 0

identity_matrix = make_matrix(5, 5, is_diagonal)
identity_matrix

[[1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1]]

Matrices will be important to us for several reasons:  
1. We can use a matrix to represent a data set consisting of several multiple vectors, simply by considering each vector as a row of the matrix.  
2. We can use an n by k matrix to represent a linear function that maps k-dimensional vectors to n-dimensional vectors.  
3. Matrices can be used to represent binary relationships.  

In Chapter 1, we represented the edges os a network as a collection of pairs (i, j).  
An alternative representation would be to create a matrix A such that <code>A[i][j]</code> is 1 if nodes i and j are connected and 0 otherwise.

In [148]:
# Recall that before we had:
friendships = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4),
               (4, 5), (5, 6), (5, 7), (6, 8), (7, 8), (8, 9)]

# We could also represent this as:
     #     user 0  1  2  3  4  5  6  7  8  9
     #
friendships = [[0, 1, 1, 0, 0, 0, 0, 0, 0, 0], # user 0
               [1, 0, 1, 1, 0, 0, 0, 0, 0, 0], # user 1
               [1, 1, 0, 1, 0, 0, 0, 0, 0, 0], # user 2
               [0, 1, 1, 0, 1, 0, 0, 0, 0, 0], # user 3
               [0, 0, 0, 1, 0, 1, 0, 0, 0, 0], # user 4
               [0, 0, 0, 0, 1, 0, 1, 1, 0, 0], # user 5
               [0, 0, 0, 0, 0, 1, 0, 0, 1, 0], # user 6
               [0, 0, 0, 0, 0, 1, 0, 0, 1, 0], # user 7
               [0, 0, 0, 0, 0, 0, 1, 1, 0, 1], # user 8
               [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]] # user 9

If there are very few connections, this is a much more inefficient representation, since you end up having to store a lot of zeroes.  
However, with the matrix representation it is much quicker to check whether two nodes are connected.  
Instead of inspecting every edge, you just do a matrix lookup.

In [149]:
print(friendships[0][2] == 1)  # True, 0 and 2 are friends 
print(friendships[0][8] == 1)  # False, 0 and 8 are not friends

True
False


Similarly, to find the connections a node has, you only need to inspect the column (or row) corresponding to that node:

In [150]:
# we only need to look at one row to find five's friends
friends_of_five = [i for i, is_friend in enumerate(friendships[5]) if is_friend]
friends_of_five

[4, 6, 7]

NOTE:  
All of the machinery we built here you get for free if you use [NumPy](http://www.numpy.org/). (You get a lot more too.) 